### Script para o Fechamento

Tratar os dados de fechamento para realizar a carga no OneIT.  

-- Criado por Victor Gonçalves  
-- 04/01/2022

--------  

Muito CUIDADO com o mês na tabela final, se não substitui tudo no ONEIT;  
A chave é o mês, se subir 1 dia do mês de 1 cliente, substitui todo aquele mês;


Sempre SUBIR SOMENTE O FECHADO/ SOMENTE PREVIA (Depende qual estiver fazendo);
1. Da coluna A>J não pode ter blank.
2. Estoque e SELL OUT trocar ZERO > Blank;
3. FAZER UM PROCV COM O MASTERDATA IBP;    **Realizado no EXCEL**  
	00. POWER BI\00. PBIX\000. DE-PARA  
4. Realizar Dinamica para retirar itens repetidos.
5. COLUNAS  - Sempre Title;

#### SCHEMA
Subir a tabela preenchida, para que o script faça realize o ETL.

Periodo | CNPJ | Modelo | Loja | Varejo | Bandeira | Cidade | Estado | GrupoProduto | EAN | QtVenda| QtEstoque  | QtDiasEstoque


In [38]:
import pandas as pd
import numpy as np

In [8]:
df_inicial = pd.read_excel("Template Input - VScode.xlsx")

In [3]:
# Não sera necessario
# LISTA DE CATEGORIAS

GrupoProduto = ['Adega','Air Fryer','Ar Condicionado','Aspirador','Batedeira','Bebedouro Água','Cafeteira',
'Cervejeira','Chaleira','Climatizador','Coifa','Cooktop','Depurador','Ferro','Fogão',
'Forno de Bancada','Forno de Embutir','Freezer Horizontal','Freezer Vertical','Front Load',
'Lavadora de Alta Pressão','Lavadora de Roupas','Lava-Louças','Liquidificador','Micro-ondas','Mixer',
'Panela Elétrica','Processador de Alimentos','Purificador de Água','Refrigerador','Secadora','Torradeira',
'Umidificador de Ar','Vaporizador de Pisos']

In [70]:
a = df_inicial['CNPJ'].unique()
a.sort()
a
# #----------
# df_inicial.info()
# #----------
# df_inicial.columns

array(['Allied', 'Amazon', 'Angeloni', 'B2W', 'Becker', 'Bemol', 'Benoit',
       'Berlanda', 'Big', 'Camicado', 'Canal Direto', 'Carrefour',
       'Carrefour.com', 'Casa E Video', 'Claudino', 'Colombo',
       'Colombo.com', 'Comprebel', 'Condor', 'Domestilar',
       'Eletro Matheus', 'Eletrosom', 'Eletrozema', 'Estrela',
       'Fast Shop', 'Fast Shop.com', 'Formosa', 'Frio Peças',
       'G. Barbosa', 'Gazin', 'Havan', 'Horfran', 'Império', 'Info Store',
       'Infoar', 'Koerich', 'Leroy Merlin', 'Lojas Cem', 'Lojas Guaibim',
       'Líder', 'Magazine Liliani', 'Magazine Luiza',
       'Magazine Luiza.com', 'Martinello', 'Martins', 'Master',
       'Mercado Livre', 'Mercadomóveis', 'Millena', 'Moveis Linhares',
       'Muffato', 'Nosso Lar', 'Novo Mundo', 'Novo Mundo.com',
       'Pernambucanas', 'Pintos', 'Polishop', 'Quero-Quero', 'Ramsons',
       'Rcell', 'Sams Club', 'Simonetti', 'Sipolatti', 'Tv Lar',
       'Venda Direta', 'Via Varejo', 'Via Varejo.com', 'Zenir'],
      dt

In [35]:
#AJUSTANDO O NOME DOS VAREJOS

col = ['CNPJ','Loja', 'Varejo', 'Bandeira','Estado']

for i in col:
    df_inicial[i] = df_inicial[i].str.title()
    df_inicial[i] = df_inicial[i].str.replace('.Com','.com')
    df_inicial[i] = df_inicial[i].str.replace('Friopeças','Frio Peças')
    # df_inicial[i] = df_inicial[i].str.replace('COOP','Cooperativa de Consumo...')


C:\Users\goncavic01\AppData\Local\Temp\ipykernel_14800\240048291.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_inicial[i] = df_inicial[i].str.replace('.Com','.com')


In [47]:
#DROP COLUNA MASTER DATA
df_inicial = df_inicial.drop(['CHECK_IBP'],axis=1)
# df_inicial.columns

KeyError: "['CHECK_IBP'] not found in axis"

In [48]:
df_inicial.columns

Index(['Periodo', 'CNPJ', 'Modelo', 'Loja', 'Varejo', 'Bandeira', 'Cidade',
       'Estado', 'GrupoProduto', 'EAN', 'QtVenda', 'QtEstoque',
       'QtDiasEstoque'],
      dtype='object')

In [66]:
#AGRUPANDO OS VALORES. 
# as_index = False permite repetir valores
# .agg > permite um dicionario das colunas para função.

df_final = df_inicial.groupby(['Periodo', 'CNPJ', 'Modelo', 'Loja', 'Varejo', 'Bandeira',
 'Cidade','Estado', 'GrupoProduto','EAN'], as_index = False).agg({'QtVenda': 'sum', 'QtEstoque': 'sum'})
 
#  ['QtVenda'].sum()

In [67]:
#TROCAR ZEROS POR NAN
df_final['QtVenda'].replace(0,np.nan, inplace=True)
df_final['QtEstoque'].replace(0,np.nan, inplace=True)

In [68]:
#AJUSTANDO COLUNA DE DATA
df_final['Periodo'] = df_final['Periodo'].dt.strftime('%d/%m/%Y')

In [69]:
# CONFERIR TAMANHO DO DF_FINAL
display(df_final)

,Periodo,CNPJ,Modelo,Loja,Varejo,Bandeira,Cidade,Estado,GrupoProduto,EAN,QtVenda,QtEstoque
0,01/12/2022,Allied,AQP20,Allied,Allied,Allied,ELECTROLUX,Allied,Aspirador,AQP20,1.0,38.0
1,01/12/2022,Allied,AWD01,Allied,Allied,Allied,ELECTROLUX,Allied,Aspirador,AWD01,NaN,3.0
2,01/12/2022,Allied,BLP70,Allied,Allied,Allied,ELECTROLUX,Allied,Liquidificador,BLP70,1.0,NaN
3,01/12/2022,Allied,CMP70,Allied,Allied,Allied,ELECTROLUX,Allied,Cafeteira,CMP70,4.0,NaN
4,01/12/2022,Allied,EAF10,Allied,Allied,Allied,ELECTROLUX,Allied,Air Fryer,EAF10,6.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6220,01/12/2022,Zenir,TC56S,Zenir,Zenir,Zenir,ELECTROLUX,Zenir,Refrigerador,TC56S,7.0,7.0
6221,01/12/2022,Zenir,TF39,Zenir,Zenir,Zenir,ELECTROLUX,Zenir,Refrigerador,TF39,27.0,353.0
6222,01/12/2022,Zenir,TF39S,Zenir,Zenir,Zenir,ELECTROLUX,Zenir,Refrigerador,TF39S,7.0,79.0
6223,01/12/2022,Zenir,TF55,Zenir,Zenir,Zenir,ELECTROLUX,Zenir,Refrigerador,TF55,9.0,44.0


In [72]:
df_final.to_excel('Template_Final_ONEIT.xlsx', index= False)